# Media Matching Bot - JarBot (arzwiki)

[T275888](https://phabricator.wikimedia.org/T275888)

## Purpose

Structured data team are working on an API that will allow bot writers to automatically add highly relevant images to specific articles.

In order to undersrand how and whether to continue to move forward or if any major changes need to be made, we would like to collect metrics about the health of the project:

- How many edits are made by bots to add images?
- What proportion of those edits are reverted within 48 hours (aka “unconstructive edits”)? 
- How many images are added to an article in each edit?
- Are there certain topic areas where images added by bots are more likely to be reverted

In [1]:
import datetime as dt
import re

import pandas as pd
import numpy as np

from wmfdata import spark

You are using wmfdata v1.3.1, but v1.3.3 is available.

To update, run `pip install --upgrade git+https://github.com/wikimedia/wmfdata-python.git@release --ignore-installed`.

To see the changes, refer to https://github.com/wikimedia/wmfdata-python/blob/release/CHANGELOG.md


In [4]:
spark_session = spark.get_session(app_name='pyspark regular; media-changes',
                                  type='yarn-large', # local, yarn-regular, yarn-large
                                  )  

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


For this analysis, we are collecting metrics for JarBot running on Egyptian Arabic Wikipedia from April 2022 to May 2022.

In [5]:
mw_snapshot = '2022-05'  
wiki_db = 'arzwiki'

start_timestamp = '2022-04-01' 
end_timestamp = '2022-06-01'

bot_name = 'JarBot'
media_list_table = 'cchen.arz_jarbot_edits'

## Data Collection

In [6]:
MEDIA_PREFIXES = ['File', 'Image', 'Media']
MEDIA_ALIASES = {"arz": ["ميديا", "صورة", "وسائط", "ملف"]}

# https://commons.wikimedia.org/wiki/Commons:File_types
IMAGE_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.svg', '.gif','.tif', '.bmp', '.webp', '.xcf','.djvu', '.pdf']
VIDEO_EXTENSIONS = ['.ogv', '.webm', '.mpg', '.mpeg']
AUDIO_EXTENSIONS = ['.ogg', '.mp3', '.mid', '.webm', '.flac', '.wav', '.oga']
MEDIA_EXTENSIONS = list(set(IMAGE_EXTENSIONS + VIDEO_EXTENSIONS + AUDIO_EXTENSIONS))

In [7]:
exten_regex = ('(' + '|'.join([e + '\\b' for e in MEDIA_EXTENSIONS]) + ')').replace('.', '\.')
extension_pattern = re.compile(f'([\w ,\(\)\.-]+){exten_regex}', flags=re.UNICODE)
bracket_pattern = re.compile('(?<=\[\[)(.*?)(?=\]\])', flags=re.DOTALL)

# NOTE: I explored several approaches to this function and how they impacted speed:
# * mwparserfromhell parsing substantially increases processing time, even compared to many regexes
# * Reducing down the number of extensions considered has a very minimal impact on time
# * Removing the first regex that extracts links has a very minimal impact on time. In theory it should be mostly unnecessary but will catch some rare file extensions.
# * Ignoring upper-case file extensions (e.g., .JPG) by not lower-casing the wikitext and just doing .findall over the iterative .search has very little impact on time

def getMedia(wikitext, wiki_db='enwiki', max_link_length=240):
    """Gather counts of media files found directly in wikitext.
    
    See https://phabricator.wikimedia.org/T299712 for more details.
    Link length: https://commons.wikimedia.org/wiki/Commons:File_naming#Length
    """
    lang = wiki_db.replace('wiki', '')
    try:
        # find standard bracket-syntax links -- this likely could be dropped but adds minimal overhead
        med_prefixes = MEDIA_PREFIXES + MEDIA_ALIASES.get(lang, [])
        links = bracket_pattern.findall(wikitext)
        bracket_links = set([l.split(':', maxsplit=1)[1].split('|', maxsplit=1)[0].strip() for l in links if l.split(':', maxsplit=1)[0] in med_prefixes])
        
        # supplement with links outside brackets as determined via known file extensions
        # lower-case to handle e.g., .JPG instead of .jpg when searching for file extensions
        lc_wt = wikitext.lower()
        exten_links = []
        end = 0
        while True:
            m = extension_pattern.search(lc_wt, pos=end)
            if m is None:
                break
            start, end = m.span()
            exten_links.append(wikitext[start:end].strip())
        return [l.replace('\n', ' ') for l in bracket_links.union(exten_links) if len(l) <= max_link_length]
    except Exception:
        return None
    
spark_session.udf.register('getMedia', getMedia, 'ARRAY<String>')

<function __main__.getMedia(wikitext, wiki_db='enwiki', max_link_length=240)>

In [8]:
def compareMediaLists(curr_media, prev_media):
    """Compare two media lists to determine what changed."""
    try:
        changes = []
        unaligned = set(curr_media) ^ set(prev_media)
        for m in unaligned:
            if m in curr_media:
                changes.append((m, 1))
            elif m in prev_media:
                changes.append((m, -1))
        return changes
    except Exception:
        return None
    
spark_session.udf.register('compareMediaLists', compareMediaLists, 'ARRAY<STRUCT<filename:STRING, action:INT>>')

<function __main__.compareMediaLists(curr_media, prev_media)>

In [9]:
## Creare media list

create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {media_list_table} (
        wiki_db                         STRING         COMMENT 'Wiki -- e.g., enwiki for English',
        page_id                         INT            COMMENT 'Article page ID',
        event_timestamp                 TIMESTAMP      ,
        revision_id                     BIGINT         COMMENT 'Revision ID',
        revision_parent_id              BIGINT         COMMENT 'Revision ID of parent revision',
        revision_is_identity_reverted   BOOLEAN        COMMENT 'Was revision reverted?',
        revision_is_identity_revert     BOOLEAN        COMMENT 'Did revision restore a previous revision?',
        revision_seconds_to_identity_revert    BIGINT        COMMENT 'seconds elapsed between revision posting and its revert',
        revision_tags                   ARRAY<STRING>  COMMENT 'Edit tags associated with revision',
        cur_rev_media_array             ARRAY<STRING>  COMMENT 'List of images in current revision',
        par_rev_media_array             ARRAY<STRING>  COMMENT 'List of images in parent revision'
    )
    """

spark_session.sql(create_table_query)

DataFrame[]

In [10]:
media_list_query  = """

WITH revisions AS (
    SELECT
      wiki_db,
      page_id,
      event_timestamp,
      revision_id,
      revision_parent_id,
      revision_is_identity_reverted,
      revision_is_identity_revert,
      revision_seconds_to_identity_revert,
      revision_tags
    FROM wmf.mediawiki_history
    WHERE
      snapshot = '{mw_snapshot}'
      AND wiki_db = '{wiki_db}'
      AND page_namespace = 0
      AND event_type = 'create'
      AND event_entity = 'revision'
       AND event_timestamp >= '{start_timestamp}'
      AND event_timestamp < '{end_timestamp}'
      AND event_user_text = '{bot_name}'
      AND NOT page_is_redirect
),
all_revision_ids AS (
    SELECT DISTINCT
      wiki_db,
      rev_id
    FROM (
        SELECT
          wiki_db,
          revision_id AS rev_id
        FROM revisions
        UNION ALL
        SELECT
          wiki_db,
          revision_parent_id AS rev_id
        FROM revisions
    ) r
),
media_lists AS (
    SELECT
      r.wiki_db,
      r.rev_id,
      getMedia(revision_text, wt.wiki_db) AS media_array
    FROM wmf.mediawiki_wikitext_history wt
    INNER JOIN all_revision_ids r
      ON (wt.wiki_db = r.wiki_db
          AND wt.revision_id = r.rev_id)
    WHERE
      snapshot = '{mw_snapshot}'
      AND wt.wiki_db = '{wiki_db}'
)

INSERT OVERWRITE TABLE {media_list_table}     
SELECT
  r.wiki_db,
  page_id,
  event_timestamp,
  revision_id,
  revision_parent_id,
  revision_is_identity_reverted,
  revision_is_identity_revert,
  revision_seconds_to_identity_revert,
  revision_tags,
  c.media_array AS cur_rev_media_array,
  p.media_array AS par_rev_media_array
FROM revisions r
LEFT JOIN media_lists c
  ON (r.wiki_db = c.wiki_db
      AND r.revision_id = c.rev_id)
LEFT JOIN media_lists p
  ON (r.wiki_db = p.wiki_db
      AND r.revision_parent_id = p.rev_id)
"""

In [11]:
data = spark.run(media_list_query.format(
            mw_snapshot = mw_snapshot,
            start_timestamp = start_timestamp,
            end_timestamp = end_timestamp ,
            wiki_db = wiki_db,
            bot_name = bot_name,
            media_list_table = media_list_table ))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
22/06/20 00:23:30 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


## Edits Count

In [12]:
edits_count_query = '''
SELECT 
  SUBSTR(event_timestamp,0,11) AS `date`,
  COUNT(DISTINCT(revision_id)) AS rev
FROM {media_list_table}
GROUP BY SUBSTR(event_timestamp,0,11)
'''

In [13]:
edits_count = spark.run(edits_count_query.format(
                media_list_table = media_list_table))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [14]:
edits_count

,date,rev
0,2022-05-03,32718
1,2022-04-25,6
2,2022-04-23,557


In [15]:
#total image edits count
edits_count['rev'].sum()

33281

## Reverted edits

In [16]:
reverted_query = '''
SELECT 
  event_timestamp,
  revision_id, 
  CASE 
    WHEN revision_is_identity_reverted 
        AND revision_seconds_to_identity_revert <= 172800 THEN 1
    ELSE 0 
  END AS reverted
FROM {media_list_table}
'''

In [17]:
edits_revert = spark.run(reverted_query.format(
                media_list_table = media_list_table))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [18]:
#numnber of image edits reverted within 48 hours
edits_revert['reverted'].sum()

3

In [19]:
#proportion
edits_revert['reverted'].sum()/edits_revert['reverted'].count()

9.014152218983804e-05

The proportion of image edits are reverted within 48 hours is 0.009%. As a reference, the 48-hour revert rate is 2.3% for overall edits and 0.3% for bot edits in Egyptian Arabic Wikipedia.

## Number of  images per edit

In [20]:
image_edits_query = """ 

SELECT
      wiki_db,
      page_id,
      revision_id,
      IF(revision_is_identity_reverted AND revision_seconds_to_identity_revert <= 172800 , TRUE, FALSE) AS was_reverted,
      INLINE(compareMediaLists(cur_rev_media_array, par_rev_media_array))
FROM {media_list_table} m
WHERE
     revision_id IS NOT NULL

"""

In [21]:
image_edits = spark.run(image_edits_query.format(
                media_list_table = media_list_table))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [24]:
image_edits_sdf = spark_session.createDataFrame(image_edits)
image_edits_sdf.createGlobalTempView("image_edits_temp")

In [25]:
image_count_query = """
SELECT revision_id,COUNT(*) AS image_count
FROM global_temp.image_edits_temp
WHERE action = 1
GROUP BY revision_id
"""

In [26]:
image_count = spark.run(image_count_query)

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [27]:
image_count ['image_count'].sum()

34687

In total 34,687 images were added by JarBot on Egyptian Arabic Wikipedia.

In [28]:
image_count.groupby(['image_count']).agg({'revision_id':'size'}).assign(pct_total=lambda x: x / x.sum()).rename(columns={'revision_id':'edit_count'})


,edit_count,pct_total
image_count,,
1,31873,0.957722
2,1407,0.042278


95.8% of the image edits by JarBot add 1 image to an article in each edits. 

## Article topics and image edits

Note that one article may have multiple topics. We are counting edits and reverts per article topic. When topics are aggregated, this results in double counting of articles and makes the totals edits and reverts look much bigger than they are.

In [30]:
topic_query = '''
SELECT 
  revision_id, 
  b.page_id,
  ato.topic,
  tc.main_topic, 
  tc.sub_topic,
  was_reverted
FROM global_temp.image_edits_temp b
INNER JOIN isaacj.article_topics_outlinks_2021_11 ato ON (
    ato.wiki_db =  'arzwiki'
    AND b.page_id = ato.pageid
    AND ato.score >= 0.5
  )
LEFT JOIN cchen.topic_component tc ON ato.topic = tc.topic
'''

In [31]:
edits_topic = spark.run(topic_query)

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [32]:
edits_topic.groupby(['main_topic']).agg({'revision_id':'size'}).assign(pct_total=lambda x: x / x.sum()).rename(columns={'revision_id':'edit_count'}).sort_values(['edit_count'],ascending=False)

,edit_count,pct_total
main_topic,,
Geography,46665,0.499946
Culture,38074,0.407907
STEM,6027,0.064570
History_and_Society,2574,0.027577


In [33]:
edits_topic.groupby(['topic']).agg({'revision_id':'size'}).assign(pct_total=lambda x: x / x.sum()).rename(columns={'revision_id':'edit_count'}).sort_values(['edit_count'],ascending=False).head(10)


,edit_count,pct_total
topic,,
Culture.Biography.Biography*,20845,0.223323
Geography.Regions.Europe.Europe*,13446,0.144054
Geography.Regions.Asia.Asia*,5766,0.061774
Geography.Geographical,4894,0.052432
Geography.Regions.Europe.Western_Europe,4759,0.050986
Culture.Sports,4107,0.044000
Geography.Regions.Europe.Eastern_Europe,3442,0.036876
Geography.Regions.Asia.West_Asia,3351,0.035901
Culture.Visual_arts.Visual_arts*,3249,0.034808


The image edits by JarBot were made across all 64 topics (please refer to [the taxonomy](https://www.mediawiki.org/wiki/ORES/Articletopic) for a detailed list of article topics). The most edited main topic is Geography. 

In [36]:
summary_m = edits_topic.groupby(['main_topic']).agg({'revision_id':'size','was_reverted':'sum'}).rename(columns={'revision_id':'edit_count'})
summary_m["revert_rate"] = summary_m['was_reverted']/summary_m['edit_count']
summary_m.sort_values(['revert_rate'],ascending=False)

,edit_count,was_reverted,revert_rate
main_topic,,,
Culture,38074,7,0.000184
Geography,46665,3,0.000064
History_and_Society,2574,0,0.000000
STEM,6027,0,0.000000


The image edits by JarBot are more likely to be reverted in article with main topic Culture, followed by Geography topic.

In [38]:
summary_t = edits_topic.groupby(['topic']).agg({'revision_id':'size','was_reverted':'sum'}).rename(columns={'revision_id':'edit_count'})
summary_t["revert_rate"] = summary_t['was_reverted']/summary_t['edit_count']
summary_t.sort_values(['was_reverted'],ascending=False).head(10)

,edit_count,was_reverted,revert_rate
topic,,,
Culture.Biography.Biography*,20845,3,0.000144
Culture.Philosophy_and_religion,1437,2,0.001392
Culture.Media.Films,811,1,0.001233
Geography.Regions.Europe.Western_Europe,4759,1,0.000210
Geography.Regions.Europe.Europe*,13446,1,0.000074
Culture.Media.Media*,2663,1,0.000376
Geography.Regions.Asia.Asia*,5766,1,0.000173
Culture.Media.Entertainment,129,0,0.000000
Culture.Media.Books,2,0,0.000000
